# Южный Парк

Импортирую всякие модули

In [35]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

Вытаскиваю и нормирую данные.

In [4]:
SO_PARK = pandas.read_csv('All-seasons.csv', sep=',')
SO_PARK.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Под главными персонажами я поняла четвёрку главных.

In [5]:
stan = SO_PARK[SO_PARK['Character'] == 'Stan']
kyle = SO_PARK[SO_PARK['Character'] == 'Kyle']
eric = SO_PARK[SO_PARK['Character'] == 'Cartman']
kenny = SO_PARK[SO_PARK['Character'] == 'Kenny']

In [6]:
print('stan: ' + str(len(stan)))
print('kyle: ' + str(len(kyle)))
print('eric: ' + str(len(eric)))
print('kenny: ' + str(len(kenny)))

stan: 7680
kyle: 7099
eric: 9774
kenny: 881


Данные несбалансированы, потому что Кенни говорит слишком мало. При этом данных очень много, потому что это все сезоны. Поэтому я, пожалуй, просто выкину часть лишних данных.

In [7]:
less_stan = stan.sample(n=len(kenny))
less_kyle = kyle.sample(n=len(kenny))
less_eric = eric.sample(n=len(kenny))
balanced = pandas.concat([less_stan, less_kyle, less_eric, kenny])
balanced.groupby('Character').describe()

Episode       Line Season
Character                                 
Cartman   count      881        881    881
          unique      18        868     18
          top          3    What?\n      3
          freq        79         10     76
Kenny     count      881        881    881
          unique      18        754     18
          top          3  (Yeah!)\n      3
          freq       106         17    132
Kyle      count      881        881    881
          unique      18        848     18
          top          1    What?\n      2
          freq        77          8    118
Stan      count      881        881    881
          unique      18        849     18
          top          6    What?\n      2
          freq        70         10     97

Теперь датасет сбалансирован.

## Baseline

In [73]:
import random

rand_names = ['Cartman', 'Kenny', 'Kyle', 'Stan'] * 881
random.shuffle(rand_names)
print(rand_names[:100])

['Kenny', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kenny', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Kenny', 'Cartman', 'Kenny', 'Kyle', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kyle', 'Kyle', 'Kyle', 'Kenny', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Cartman', 'Stan', 'Cartman', 'Stan', 'Kyle', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Stan', 'Stan', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Kyle', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Cartman', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kyle', 'Kenny', 'Kenny', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kyle', 'Kenny', 'Kenny', 'Cartman', 'Stan', 'Stan', 'Cartman', 'Kenny']


In [74]:
print(classification_report(balanced['Character'], rand_names))

             precision    recall  f1-score   support

    Cartman       0.27      0.27      0.27       881
      Kenny       0.27      0.27      0.27       881
       Kyle       0.25      0.25      0.25       881
       Stan       0.25      0.25      0.25       881

avg / total       0.26      0.26      0.26      3524



Если распределять героев с рандомно с одинаковой вероятностью, точность в райное 0.25, что полностью соответствует теории вероятностей.

## Preparing data

Дальше я попробую лемматизировать и не лемматизировать. Лемматизировать – потому что это кажется разумным решением, в конце концов, о человеке должны больше говорять лексемы, которые он употребляет, чем словоформы. Не лемматизировать, потому что в прошлом дз в задаче классификации писем внезапно лучше показали семя не лемматизированные словари.

In [8]:
# не лемматизируя
bow_non_lem = CountVectorizer()
bow_non_lem.fit_transform(balanced['Line'])
bowed_non_lem = bow_non_lem.transform(balanced['Line'])

In [9]:
# функция для лемматизации. коммент: я пыталась сделать это как полагается, и сначала сделать POS-tagging,
# но лемматизатор ворднета не идеален. он, например, вообще не умеет лемматизировать местоимения...

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def lemmatize(word):
    noun = lmtzr.lemmatize(word)
    verb = lmtzr.lemmatize(word, 'v')
    if verb != word:
        return verb
    return noun

def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text if word not in '!?,.":;']

bow_lem = CountVectorizer(tokenizer=tokenize_lemmatize)
bow_lem.fit_transform(balanced['Line'])
bowed_lem = bow_lem.transform(balanced['Line'])

In [ ]:
Смотрю на частотные словари.

In [17]:
d = bow_non_lem.vocabulary_
srtng = []
for key in d:
    srtng.append((key, d[key]))
sorted(srtng, key=lambda x: -x[1])

[('zombies', 3754),
 ('zombie', 3753),
 ('ziplining', 3752),
 ('zipline', 3751),
 ('zero', 3750),
 ('yyeah', 3749),
 ('yup', 3748),
 ('yummy', 3747),
 ('yuhwell', 3746),
 ('yuh', 3745),
 ('yow', 3744),
 ('youuu', 3743),
 ('youtube', 3742),
 ('youth', 3741),
 ('yourselves', 3740),
 ('yourself', 3739),
 ('yours', 3738),
 ('your', 3737),
 ('younger', 3736),
 ('young', 3735),
 ('you', 3734),
 ('yothers', 3733),
 ('yo', 3732),
 ('yikes', 3731),
 ('yet', 3730),
 ('yesterday', 3729),
 ('yes', 3728),
 ('yep', 3727),
 ('yellow', 3726),
 ('yelled', 3725),
 ('yehes', 3724),
 ('yeh', 3723),
 ('yeeeeehah', 3722),
 ('yeeaahh', 3721),
 ('yee', 3720),
 ('years', 3719),
 ('year', 3718),
 ('yeahyeahyeah', 3717),
 ('yeahhh', 3716),
 ('yeahah', 3715),
 ('yeah', 3714),
 ('yea', 3713),
 ('ye', 3712),
 ('yay', 3711),
 ('yards', 3710),
 ('yard', 3709),
 ('yahtzee', 3708),
 ('yahoo', 3707),
 ('yadadadadee', 3706),
 ('yaaayaya', 3705),
 ('ya', 3704),
 ('xbox', 3703),
 ('wwwhy', 3702),
 ('wuss', 3701),
 ('wrote'

In [19]:
d = bow_lem.vocabulary_
srtng = []
for key in d:
    srtng.append((key, d[key]))
sorted(srtng, key=lambda x: -x[1])

[('…so', 3374),
 ('…hey', 3373),
 ('}', 3372),
 ('{', 3371),
 ('zombie', 3370),
 ('ziplining', 3369),
 ('zipline', 3368),
 ('zero', 3367),
 ('yyeah', 3366),
 ('yup', 3365),
 ('yummy', 3364),
 ('yuhwell', 3363),
 ('yuh-eh-you', 3362),
 ('yuh', 3361),
 ('yow', 3360),
 ('youuu', 3359),
 ('youtube', 3358),
 ('youth', 3357),
 ('yourselves', 3356),
 ('yourself', 3355),
 ('yours', 3354),
 ('your', 3353),
 ('younger', 3352),
 ('young', 3351),
 ('you-you', 3350),
 ('you-hoo', 3349),
 ('you-', 3348),
 ('you', 3347),
 ('yothers', 3346),
 ('yo', 3345),
 ('yet', 3344),
 ('yes…', 3343),
 ('yesterday', 3342),
 ('yes', 3341),
 ('yep', 3340),
 ('yellow', 3339),
 ('yell', 3338),
 ('yehes', 3337),
 ('yeh', 3336),
 ('yeeeeehah', 3335),
 ('yeeaahh', 3334),
 ('yee', 3333),
 ('year', 3332),
 ('yeahyeahyeah', 3331),
 ('yeahhh', 3330),
 ('yeahah', 3329),
 ('yeah-uh', 3328),
 ('yeah', 3327),
 ('yea', 3326),
 ('ye-yes', 3325),
 ('ye-but', 3324),
 ('yay', 3323),
 ('yard', 3322),
 ('yahtzee', 3321),
 ('yahoo', 332

Мне кажется, что с лемматизацией будет работать лучше, потому что первые же два слова нелемматизированного словаря – две словоформы одной леммы.

## Training and testing

In [21]:
# (криво) лематизированные реплики
lem_train, lem_test, y_train_lem, y_test_lem = train_test_split(bowed_lem, 
                                                    balanced['Character'], 
                                                    test_size=0.2)
print(lem_train.shape, lem_test.shape, y_train_lem.shape, y_test_lem.shape)

(2819, 3375) (705, 3375) (2819,) (705,)


In [22]:
# не лемматизированные реплики
nlem_train, nlem_test, y_train_nlem, y_test_nlem = train_test_split(bowed_non_lem, 
                                                    balanced['Character'], 
                                                    test_size=0.2)
print(nlem_train.shape, nlem_test.shape, y_train_nlem.shape, y_test_nlem.shape)

(2819, 3755) (705, 3755) (2819,) (705,)


### Рандомный лес

Сначала я делаю это с дефольными параметрами.

In [27]:
# на лемматизированных данных
forest_lem = RandomForestClassifier()
forest_lem.fit(lem_train, y_train_lem)
predict_y = forest_lem.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.41      0.31      0.35       173
      Kenny       1.00      0.94      0.97       202
       Kyle       0.39      0.41      0.40       169
       Stan       0.31      0.39      0.34       161

avg / total       0.55      0.53      0.54       705



In [26]:
# на нелемматизированных данных
forest_nlem = RandomForestClassifier(n_estimators = 100)
forest_nlem.fit(nlem_train, y_train_nlem)
predict_y = forest_nlem.predict(nlem_test)
print(classification_report(y_test_nlem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.42      0.40      0.40       167
      Kenny       0.52      0.45      0.48       178
       Kyle       0.35      0.40      0.37       171
       Stan       0.32      0.33      0.33       189

avg / total       0.40      0.39      0.40       705



Действительно, первый словарь работает лучше.

Теперь варьирую параметры.

In [29]:
# на лемматизированных данных, т.к. они показали лучший результат
forest_lem = RandomForestClassifier()
n_estimators = list(np.arange(1, 121, 10))
max_features = [None, 'sqrt', 'log2']

parameters = {'n_estimators': n_estimators, 'max_features': max_features}
grid = GridSearchCV(forest_lem, parameters)
grid.fit(lem_train, y_train_lem)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111], 'max_features': [None, 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [31]:
print(grid.best_score_)

0.559063497694


In [32]:
grid.best_params_

{'max_features': None, 'n_estimators': 91}

In [33]:
forest_best = grid.best_estimator_

### Наивный Байес 

In [36]:
naive_model = MultinomialNB()
naive_model.fit(lem_train, y_train_lem)
predict_y = naive_model.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.43      0.44      0.43       173
      Kenny       0.83      0.95      0.89       202
       Kyle       0.38      0.33      0.35       169
       Stan       0.37      0.35      0.36       161

avg / total       0.52      0.54      0.53       705



In [38]:
naive_model = MultinomialNB()
naive_model.fit(nlem_train, y_train_nlem)
predict_y = naive_model.predict(nlem_test)
print(classification_report(y_test_nlem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.38      0.45      0.41       167
      Kenny       0.52      0.45      0.48       178
       Kyle       0.30      0.33      0.32       171
       Stan       0.38      0.34      0.36       189

avg / total       0.40      0.39      0.39       705



Тут тоже с лемматизированными лучше.

In [40]:
naive_model = MultinomialNB()
alpha = 0.01, 1, 10, 100
fit_prior = True, False

parameters = {'alpha': alpha, 'fit_prior': fit_prior}
grid = GridSearchCV(naive_model, parameters)
grid.fit(lem_train, y_train_lem)

GridSearchCV(cv=None, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': (0.01, 1, 10, 100), 'fit_prior': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [41]:
print(grid.best_score_)

0.523589925505


In [42]:
grid.best_params_

{'alpha': 1, 'fit_prior': False}

In [43]:
nb_best = grid.best_estimator_

### Логит регрессия

In [47]:
log_regr = LogisticRegression()
log_regr.fit(lem_train, y_train_lem)
predict_y = log_regr.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.44      0.46      0.45       173
      Kenny       0.99      0.92      0.95       202
       Kyle       0.38      0.39      0.38       169
       Stan       0.39      0.40      0.39       161

avg / total       0.57      0.56      0.57       705



In [48]:
log_regr = LogisticRegression()
log_regr.fit(nlem_train, y_train_nlem)
predict_y = log_regr.predict(nlem_test)
print(classification_report(y_test_nlem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.44      0.46      0.45       167
      Kenny       0.47      0.60      0.52       178
       Kyle       0.38      0.32      0.34       171
       Stan       0.38      0.31      0.34       189

avg / total       0.41      0.42      0.41       705



Да, с лемматизированными данными гораздо лучше.

In [50]:
C = [1, 10, 100, 1000]
dual = True, False

parameters = {'C': C, 'dual': dual}
grid = GridSearchCV(log_regr, parameters)
grid.fit(lem_train, y_train_lem)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'dual': (True, False), 'C': [1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [51]:
print(grid.best_score_)

0.561546647747


Это лучший результат из всех.

In [52]:
grid.best_params_

{'C': 1, 'dual': True}

In [53]:
log_best = grid.best_estimator_

## Testing

In [44]:
predict_y = forest_best.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.45      0.46      0.45       173
      Kenny       0.99      0.94      0.96       202
       Kyle       0.46      0.33      0.38       169
       Stan       0.38      0.50      0.43       161

avg / total       0.59      0.58      0.58       705



In [46]:
predict_y = nb_best.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.43      0.44      0.43       173
      Kenny       0.83      0.95      0.89       202
       Kyle       0.38      0.33      0.35       169
       Stan       0.37      0.35      0.36       161

avg / total       0.52      0.54      0.53       705



In [54]:
predict_y = log_best.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.44      0.46      0.45       173
      Kenny       0.99      0.92      0.95       202
       Kyle       0.38      0.39      0.38       169
       Stan       0.39      0.40      0.39       161

avg / total       0.57      0.56      0.57       705



Несмотря на то, что логистическая регрессия показывала лучший результат на train set, на test сет лучше всё-таки рандомный лес. Видимо, это говорит о том, что логистическая модель немного переобучилась.

## Graphs